# Data Example Geometry

Define some filenames

In [ ]:
import os
data_filename   = os.path.join(os.pardir, "data", "all_sz.json")
params_filename = os.path.join(os.pardir, "data", "default_params.json")

Load the subduction zone parameters

In [ ]:
import json
with open(data_filename, "r") as fp:
    szparams = json.load(fp)

The "minimum" resolution, used to scale resolutions throughout the mesh.  The current default of 10.0km is way too high for a real subduction zone model but helps to keep this test notebook fast and readable.

In [ ]:
minres = 10.0 # km

In [ ]:
import numpy as np

Load the default parameters

In [ ]:
with open(params_filename, "r") as fp:
    default_params = json.load(fp)

List the available subduction zones that we've just loaded from the json file

In [ ]:
print("Available subduction zones:")
for k in szparams.keys(): print(k)

## This is where you can switch between subduction zones!!

Choose a subduction zone from the list on the first line of the following cell to load the geometrical parameters from the dictionary

In [ ]:
szname = "07_Nicaragua"  # change this to switch subduction zones
szdata = szparams[szname]
coast_distance = szdata['deltaxcoast']
extra_width = szdata['deltaxwidth']
lc_depth = szdata['deltazmoho']
uc_depth = szdata['deltazuc']
io_depth = szdata['deltazoutflow']
points = np.asarray([szdata['trench']]+szdata['slabpoints'])
xs = points[:,0]
ys = points[:,1]
res = [1*minres if y >= -default_params["slab_diag_depth2"] else 3*minres for y in ys]

Load the geometry module so we can build a mesh based on the above parameters

In [ ]:
import sys, os
sys.path.append(os.path.join(os.path.pardir, 'python'))
import geometry as geo

Define the slab spline

In [ ]:
# set up the surface ids for the slab depending on depth
sids = []
for y in ys[1:]:
    if y >= -lc_depth: 
        sid = default_params['fault_sid']
    elif y >= -default_params['slab_diag_depth1']:
        sid = default_params['slab_sid']
    elif y >= -default_params['slab_diag_depth2']:
        sid = default_params['slab_diag_sid']
    else:
        sid = default_params['slab_sid']
    sids.append(sid)
sids

In [ ]:
# set up the slab spline
slab = geo.SlabSpline(xs, ys, res=res, sid=sids, name="Slab")
# adding the coupling depths may or may not be necessary
# depending on if they were included in the slab spline data already or not
# the slab class should ignore them if they aren't necessary
slab.addpoint(default_params['partial_coupling_depth'], "Slab::PartialCouplingDepth", 
              res=default_params['partial_coupling_depth_res_fact']*minres, 
              sid=default_params['slab_diag_sid'])
slab.addpoint(default_params['full_coupling_depth'], "Slab::FullCouplingDepth", 
              res=default_params['full_coupling_depth_res_fact']*minres, 
              sid=default_params['slab_diag_sid'])
# add the slab detector point
slab.addpoint(default_params["slab_det_depth"], "Slab::DetectorPoint", 
              res=default_params["full_coupling_depth_res_fact"]*minres,
              sid=default_params['slab_diag_sid'])

Define the subduction geometry, including crustal layers and plot

In [ ]:
geom = geo.SubductionGeometry(slab, **default_params, 
                              wedge_side_top_res=default_params["wedge_side_top_res_fact"]*minres,
                              wedge_side_base_res=default_params["wedge_side_base_res_fact"]*minres,
                              slab_side_base_res=default_params["slab_side_base_res_fact"]*minres,
                              coast_res=default_params["coast_res_fact"]*minres,
                              coast_distance=coast_distance, extra_width=extra_width)
# add a lower crust
geom.addcrustlayer(lc_depth, "LowerCrust", 
                   sid=default_params["lc_base_sid"], rid=default_params["lc_rid"],
                   slab_res=default_params["lc_slab_res_fact"]*minres,
                   side_res=default_params["lc_side_res_fact"]*minres,
                   slab_sid=default_params["fault_sid"],
                   side_sid=default_params["lc_side_sid"])
# oceanic subduction zones don't define an upper crust
if uc_depth is not None:
    geom.addcrustlayer(uc_depth, "UpperCrust", 
                       sid=default_params["uc_base_sid"], rid=default_params["uc_rid"],
                       slab_res=default_params["uc_slab_res_fact"]*minres,
                       side_res=default_params["uc_side_res_fact"]*minres,
                       slab_sid=default_params["fault_sid"],
                       side_sid=default_params["uc_side_sid"])
# add the pre-defined in-out point
geom.addwedgesidepoint(io_depth, "WedgeSide::InOut", line_name="UpperWedgeSide", 
                       res=default_params["io_depth_res_fact"]*minres, 
                       sid=default_params["upper_wedge_side_sid"])
# add wedge dividers for the diagnostics
geom.addwedgedivider(default_params["slab_diag_depth1"], "ColdCorner", 
                     slab_res=default_params["slab_diag_res2_fact"]*minres, 
                     top_res=default_params["slab_diag_res2_fact"]*minres,
                     rid=default_params["wedge_rid"], 
                     slab_sid=default_params["slab_sid"])
# add wedge dividers for the diagnostics
geom.addwedgedivider(default_params["slab_diag_depth2"], "WedgeFocused", 
                     slab_res=default_params["slab_diag_res1_fact"]*minres, 
                     top_res=default_params["slab_diag_res1_fact"]*minres,
                     rid=default_params["wedge_diag_rid"], 
                     slab_sid=default_params["slab_diag_sid"])
geom.plot(label_sids=True, label_rids=True)
#geom.writegeofile(szname+".geo_unrolled")

In [ ]:
mesh, ct, ft = geom.generatemesh()

In [ ]:
import pyvista
pyvista.start_xvfb()
from dolfinx.plot import create_vtk_mesh
topology, cell_types, x = create_vtk_mesh(mesh, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cell_types, x)
num_local_cells = mesh.topology.index_map(mesh.topology.dim).size_local
# grid.cell_data["Marker"] = ct.values[ct.indices<num_local_cells]
# grid.set_active_scalars("Marker")

p = pyvista.Plotter(window_size=[800, 800])
p.add_mesh(grid, show_edges=True)
p.view_xy()
p.show()